## Load functions

In [5]:
import text_processing as text 
import conversion_tools as conv 

treaty_data = pd.read_csv('treaty_data/treaty_data.csv')
treaty_data.head()

,cow,country,year,title,agrmtno,core,type,issue1,issue2,createyear,entryforceyear,signyear,ratyear,involvedIO,statereport,indcomplaint,statecomplaint
0,2,United States,1947,Convention Relating to the Status of Stateless...,10,0,Convention,Stateless/Migrant,NaN,1954,1960,-88,-88,NaN,NaN,NaN,NaN
1,2,United States,1948,Convention Relating to the Status of Stateless...,10,0,Convention,Stateless/Migrant,NaN,1954,1960,-88,-88,NaN,NaN,NaN,NaN
2,2,United States,1949,Convention Relating to the Status of Stateless...,10,0,Convention,Stateless/Migrant,NaN,1954,1960,-88,-88,NaN,NaN,NaN,NaN
3,2,United States,1950,Convention Relating to the Status of Stateless...,10,0,Convention,Stateless/Migrant,NaN,1954,1960,-88,-88,NaN,NaN,NaN,NaN
4,2,United States,1951,Convention Relating to the Status of Stateless...,10,0,Convention,Stateless/Migrant,NaN,1954,1960,-88,-88,NaN,NaN,NaN,NaN


In [6]:
upr_data = pd.read_csv('upr_data_all_sessions.csv')
upr_data.head()

/tmp/ipykernel_64259/3688110203.py:1: DtypeWarning: Columns (1,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  upr_data = pd.read_csv('upr_data_all_sessions.csv')


,Unnamed: 0,Title,Date.added,Template,Cycle,Session,State.under.Review,State.under.Review...Regional.Group,Unnamed: 8,Recommendation,...,Unnamed: 28,Issues,issue1,issue2,issue3,issue4,issue5,issue6,issue7,issue8
0,upr3.1.1,141.20.3,5/18/20,Recommendation,Cycle 3 (2017 - 2021),29/01/2018,United Arab Emirates,Asia (Asian Group),NaN,Ratify the Optional Protocol to the Convention...,...,NaN,International humanitarian law,International instruments,Rights of the Child,NaN,NaN,NaN,NaN,NaN,NaN
1,upr3.1.2,141.20.2,5/18/20,Recommendation,Cycle 3 (2017 - 2021),29/01/2018,United Arab Emirates,Asia (Asian Group),NaN,Ratify the Optional Protocol to the Convention...,...,NaN,International humanitarian law,International instruments,Rights of the Child,NaN,NaN,NaN,NaN,NaN,NaN
2,upr3.1.3,141.20.1,5/18/20,Recommendation,Cycle 3 (2017 - 2021),29/01/2018,United Arab Emirates,Asia (Asian Group),NaN,Ratify the Optional Protocol to the Convention...,...,NaN,International humanitarian law,International instruments,Rights of the Child,NaN,NaN,NaN,NaN,NaN,NaN
3,upr3.1.4,141.21.2,5/18/20,Recommendation,Cycle 3 (2017 - 2021),29/01/2018,United Arab Emirates,Asia (Asian Group),NaN,Ratify the Optional Protocol to the Convention...,...,NaN,Rights of the Child,International instruments,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,upr3.1.5,141.21.1,5/18/20,Recommendation,Cycle 3 (2017 - 2021),29/01/2018,United Arab Emirates,Asia (Asian Group),NaN,Ratify the Optional Protocol to the Convention...,...,NaN,Rights of the Child,International instruments,NaN,NaN,NaN,NaN,NaN,NaN,NaN
